# Text classification ML approach

In [1]:
# import libraries
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/media/mahfuz/Media/datasets/NLP_learn/data_preprocessing/IMDB_dataset_50k_movie_review/IMDB_Dataset.csv')

In [3]:
df = df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [6]:
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [7]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [8]:
df.duplicated().sum()

17

In [9]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

### text preprocessing

In [10]:
# lowercasing
df['review'] = df['review'].str.lower()

In [13]:
# remove tag
import re
def remove_tag(row_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', row_text)
    return cleaned_text

In [14]:
df['review'] = df['review'].apply(remove_tag)

In [15]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
9995,"fun, entertaining movie about wwii german spy ...",positive
9996,give me a break. how can anyone say that this ...,negative
9997,this movie is a bad movie. but after watching ...,negative
9998,this is a movie that was probably made to ente...,negative


In [17]:
# stop words

from nltk.corpus import stopwords
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [18]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive


## now split target value

In [19]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [20]:
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [21]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

In [23]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [24]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [26]:
X_train.shape

(7986, 1)

In [27]:
X_test.shape

(1997, 1)

In [ ]:
y_test.shape

In [28]:
y_train.shape

(7986,)

### apply Bag of Words

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
cv = CountVectorizer()

In [32]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [33]:
X_train_bow.shape

(7986, 48282)

In [34]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X_train_bow, y_train)

GaussianNB()

In [35]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(y_test, y_pred)

0.6324486730095142

In [36]:
confusion_matrix(y_test, y_pred)

array([[717, 235],
       [499, 546]])

### next algo

In [37]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow, y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test, y_pred)

0.8472709063595393

In [40]:
cv1 = CountVectorizer(max_features=4000)
X_train_bow_1 = cv1.fit_transform(X_train['review']).toarray()
X_test_bow_1 = cv1.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow_1, y_train)
y_pred = rf.predict(X_test_bow_1)
accuracy_score(y_test, y_pred)

0.8417626439659489

## n-grams

In [43]:
cv2 = CountVectorizer(ngram_range=(1,3), max_features=6000)
X_train_bow_1 = cv2.fit_transform(X_train['review']).toarray()
X_test_bow_1 = cv2.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow_1, y_train)
y_pred = rf.predict(X_test_bow_1)
accuracy_score(y_test, y_pred)

0.8467701552328493

## using tf-idf

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
tfidf = TfidfVectorizer()

In [47]:
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [48]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf, y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test, y_pred)

0.8432648973460191

# Word2Vac

In [49]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [50]:
story = []
for doc in df['review']:
    row_sent = sent_tokenize(doc)
    for sent in row_sent:
        story.append(simple_preprocess(sent))

In [51]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [53]:
model.build_vocab(story)

In [54]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5876372, 6212140)

In [55]:
len(model.wv.index_to_key)

31845